<a href="https://colab.research.google.com/github/JanChocyk/Face_in_the_window/blob/master/human_face_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Owerview
In this notebook, I build, train and test my own model that classifies the human face for use in the game "Face in the window".

I will use a pre-trained model and transfer learning technique to get a classifier for the problem of recognizing two categories: human face or something else.

I'll load the model from TF Hub, then add an output layer appropriate to our problem. Then I will train the model on the previously prepared dataset.

Let's start!

### Imports

In [2]:
import numpy as np
import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import os
print(f'Tensorflow version: {tf.__version__}')
print(f'Tensorflow Hub version: {hub.__version__}')

Tensorflow version: 2.12.0
Tensorflow Hub version: 0.14.0


### Prepare data and augmentation


In [ ]:
IMG_SIZE = 224

normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

In [6]:
data_dir = '...'

batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

### Prepare Model
- load pre-trainned model
- set output layer
- compile and summary model

In [3]:
feature_extractor_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
IMAGE_SHAPE = (224, 224)

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_url,
    input_shape=IMAGE_SHAPE+(3,)
)

# zamrażamy wagi ekstraktora
feature_extractor_layer.trainable = False

In [4]:
# budujemy model dokładając ostatnią warstwę klasyfikacji
model = tf.keras.Sequential([
  normalization_layer,
  data_augmentation,
  feature_extractor_layer,
  tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 4,052,126
Trainable params: 2,562
Non-trainable params: 4,049,564
_________________________________________________________________


### Traning model

In [ ]:
epochs = 10
dt = datetime.now().strftime('%d_%m_%Y_%H_%M')
filepath = os.path.join('output', 'model_' + dt + '.h5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=epochs,
                    callbacks = [checkpoint])

### Visualization training process

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Evaluation and prodiction

In [6]:
test_dir = '...'

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    subset="test",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

loss, accuracy = model.evaluate(test_ds)
print('Test accuracy :', accuracy)

### Test Clasiffication

In [ ]:
loaded_model = tf.keras.models.load_model(filepath)

input_data = '...'

predictions = loaded_model.predict(input_data)